In [1]:
import pandas as pd
import numpy as np
import plotly
import matplotlib.pyplot as plt
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.express as px
import plotly.figure_factory as ff
from plotly.subplots import make_subplots
import ipywidgets as widgets
from IPython.display import display
import yfinance as yf
import dash
from dash import dcc
from dash import html


import statsmodels.api as sm
import statsmodels.formula.api as smf
from linearmodels.panel import compare
from linearmodels import RandomEffects
from linearmodels import PanelOLS
from linearmodels import RandomEffects
from statsmodels.tsa.stattools import adfuller
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf

import seaborn as sns

import math

c:\Users\jespe\anaconda3\lib\site-packages\geopandas\_compat.py:124: UserWarning: The Shapely GEOS version (3.11.1-CAPI-1.17.1) is incompatible with the GEOS version PyGEOS was compiled with (3.10.4-CAPI-1.16.2). Conversions between both will be slow.
  warnings.warn(


In [2]:
df_econ = pd.read_excel('df_eco_ver1.xlsx', header=[0], index_col=0)
    

In [3]:
type = pd.Categorical(df_econ.port_id)
df_eco_multi = df_econ.set_index(['port_id', 'roll_id'])
df_eco_multi['groups'] = type

In [4]:
df_eco_copy = df_eco_multi.copy()

In [7]:
def POLS_reg(df, summary:True):
    
    #setting up variables
    y_pols = df['Return']

    X_pols_all = df[['E_Score', 'S_Score', 'G_Score']]
    
    

    X_pols_all = sm.add_constant(X_pols_all)


    pols_ret_all = sm.OLS(y_pols, X_pols_all).fit()
    
    if summary == True:
        return pols_ret_all.summary()
    else:
        return  pols_ret_all

In [9]:
POLS_reg(df_eco_copy,False).ssr

3.7093947222832164

In [10]:
df_with_dummies = df_econ.copy()

entity_dummies_ols = pd.get_dummies(df_eco_copy.index.get_level_values('port_id'), drop_first=True, prefix='port').astype(int)
df_with_dummies = df_with_dummies.join(entity_dummies_ols) 

In [11]:
df_econ_copy = df_econ.copy()
df_econ_copy

port_id  roll_id    Return   E_Score   S_Score   G_Score
0          1        1  0.318618  0.461328  0.234683  0.404895
1          1        2  0.144415  0.356724  0.088171  0.338325
2          1        3  0.195616  0.281313  0.084955  0.299799
3          1        4  0.622834  0.239875 -0.035125  0.256839
4          1        5  0.670127  0.221844  0.001027  0.256443
..       ...      ...       ...       ...       ...       ...
120        5        1  0.633014  0.787066  0.605680  0.680387
121        5        2  1.126303  0.685368  0.771440  0.705555
122        5        3  0.424647  0.819015  0.643678  0.607776
123        5        4  0.477055  0.843682  0.614896  0.593464
124        5        5  0.423460  0.867668  0.601703  0.568922

[125 rows x 6 columns]

In [18]:
def RE_model(df_eco_copy, df_econ_copy, summary:True, e:True, s:True, g:True, all:True):
    y_pols = df_econ_copy['Return']
    X_re_e = df_econ_copy[['E_Score']]
    X_re_s = df_econ_copy[['S_Score']]
    X_re_g = df_econ_copy[['G_Score']]
    X_pols_all = df_econ_copy[['E_Score', 'S_Score', 'G_Score']]
    X_re_all = df_econ_copy[['E_Score', 'S_Score', 'G_Score']]
    unit_names = ['port_1', 'port_2', 'port_3', 'port_4', 'port_5']
    n = 5
    T = 5
    N = n*T

  
    
    
    if e == True:
        k = len(X_re_e.columns) + 1

        df_with_dummies = df_econ_copy.copy()
        entity_dummies_ols = pd.get_dummies(df_eco_copy.index.get_level_values('port_id'), drop_first=True, prefix='port').astype(int)
        df_with_dummies = df_with_dummies.join(entity_dummies_ols) 
        
        lsdv_expr = 'Return' + ' ~ '
        i = 0
        for X_var_name in X_re_e:
            if i > 0:
                lsdv_expr = lsdv_expr + ' + ' + X_var_name
            else:
                lsdv_expr = lsdv_expr + X_var_name
            i = i + 1
        for dummy_name in unit_names[1:]:
            lsdv_expr = lsdv_expr + ' + ' + dummy_name
            
        lsdv_model = smf.ols(formula=lsdv_expr, data=df_with_dummies)
        lsdv_model_results = lsdv_model.fit()

        
        #Calculate sigma-square-epsilon
        sigma2_epsilon = lsdv_model_results.ssr/(n*T-(n+k+1))
        # print('sigma2_epsilon = ' + str(sigma2_epsilon))

        #Calculate sigma-square-pooled
        sigma2_pooled = sm.OLS(endog=y_pols, exog=X_re_e).fit().ssr/(n*T-(k+1))
        # print('sigma2_pooled = ' + str(sigma2_pooled))

        #Calculate sigma-square-u
        sigma2_u = sigma2_pooled - sigma2_epsilon
        # print('sigma2_u = ' + str(sigma2_u))

        #Calculate theta
        theta = 1 - math.sqrt(sigma2_epsilon/(sigma2_epsilon + T*sigma2_u))
        # print('theta = ' + str(theta))
        
        
        df_eco_copy_means = df_econ_copy.groupby('port_id').mean()

        df_eco_copy_means['const'] = 1.0
        
        # print(df_eco_copy_means)
        
        
        # #Prepare the data set for mean centering the y and X columns:
        pooled_y_with_unit_name = pd.concat([df_econ_copy['port_id'], y_pols], axis=1)
        pooled_X_with_unit_name = pd.concat([df_econ_copy['port_id'], X_re_e], axis=1)

        #Center each X value using the θ-scaled group-specific mean:
        unit_name = ''
        for row_index, row in pooled_X_with_unit_name.iterrows():
            for column_name, cell_value in row.items():
                if column_name == 'port_id':
                    unit_name = pooled_X_with_unit_name.at[row_index, column_name]
                else:
                    pooled_X_group_mean = df_eco_copy_means.loc[unit_name][column_name]
                    pooled_X_with_unit_name.at[row_index, column_name] = pooled_X_with_unit_name.at[
                        row_index, column_name] - theta*pooled_X_group_mean

        #Center each y value using the θ-scaled group-specific mean:
        for row_index, row in pooled_y_with_unit_name.iterrows():
            for column_name, cell_value in row.items():
                if column_name == 'port_id':
                    unit_name = pooled_y_with_unit_name.at[row_index, column_name]
                else:
                    pooled_y_group_mean = df_eco_copy_means.loc[unit_name][column_name]
                    pooled_y_with_unit_name.at[row_index, column_name] = pooled_y_with_unit_name.at[
                        row_index, column_name] - theta*pooled_y_group_mean
                    
        #Carve out the y and X matrices:
        re_y=pooled_y_with_unit_name[list(pooled_y_with_unit_name.columns[1:])]
        re_X=pooled_X_with_unit_name[list(pooled_X_with_unit_name.columns[1:])]
        re_X= sm.add_constant(re_X)
        #Build and train the model
        re_model_e = sm.OLS(endog=re_y, exog=re_X)
        re_model_results_e = re_model_e.fit()
        if summary == True:
            return re_model_results_e.summary()
        else:
            return re_model_results_e
    elif s== True:
                k = len(X_re_s.columns) + 1

                df_with_dummies = df_econ_copy.copy()
                entity_dummies_ols = pd.get_dummies(df_eco_copy.index.get_level_values('port_id'), drop_first=True, prefix='port').astype(int)
                df_with_dummies = df_with_dummies.join(entity_dummies_ols)
                
                lsdv_expr = 'Return' + ' ~ '
                i = 0
                for X_var_name in X_re_s:
                    if i > 0:
                        lsdv_expr = lsdv_expr + ' + ' + X_var_name
                    else:
                        lsdv_expr = lsdv_expr + X_var_name
                    i = i + 1
                for dummy_name in unit_names[1:]:
                    lsdv_expr = lsdv_expr + ' + ' + dummy_name
            
                lsdv_model = smf.ols(formula=lsdv_expr, data=df_with_dummies)
                lsdv_model_results = lsdv_model.fit()

                
                #Calculate sigma-square-epsilon
                sigma2_epsilon = lsdv_model_results.ssr/(n*T-(n+k+1))
                # print('sigma2_epsilon = ' + str(sigma2_epsilon))

                #Calculate sigma-square-pooled
                sigma2_pooled = sm.OLS(endog=y_pols, exog=X_re_s).fit().ssr/(n*T-(k+1))
                # print('sigma2_pooled = ' + str(sigma2_pooled))

                #Calculate sigma-square-u
                sigma2_u = sigma2_pooled - sigma2_epsilon
                # print('sigma2_u = ' + str(sigma2_u))

                #Calculate theta
                theta = 1 - math.sqrt(sigma2_epsilon/(sigma2_epsilon + T*sigma2_u))
                # print('theta = ' + str(theta))
                
                
                df_eco_copy_means = df_econ_copy.groupby('port_id').mean()

                df_eco_copy_means['const'] = 1.0
                
                # print(df_eco_copy_means)
                
                
                # #Prepare the data set for mean centering the y and X columns:
                pooled_y_with_unit_name = pd.concat([df_econ_copy['port_id'], y_pols], axis=1)
                pooled_X_with_unit_name = pd.concat([df_econ_copy['port_id'], X_re_s], axis=1)

                #Center each X value using the θ-scaled group-specific mean:
                unit_name = ''
                for row_index, row in pooled_X_with_unit_name.iterrows():
                    for column_name, cell_value in row.items():
                        if column_name == 'port_id':
                            unit_name = pooled_X_with_unit_name.at[row_index, column_name]
                        else:
                            pooled_X_group_mean = df_eco_copy_means.loc[unit_name][column_name]
                            pooled_X_with_unit_name.at[row_index, column_name] = pooled_X_with_unit_name.at[
                                row_index, column_name] - theta*pooled_X_group_mean

                #Center each y value using the θ-scaled group-specific mean:
                for row_index, row in pooled_y_with_unit_name.iterrows():
                    for column_name, cell_value in row.items():
                        if column_name == 'port_id':
                            unit_name = pooled_y_with_unit_name.at[row_index, column_name]
                        else:
                            pooled_y_group_mean = df_eco_copy_means.loc[unit_name][column_name]
                            pooled_y_with_unit_name.at[row_index, column_name] = pooled_y_with_unit_name.at[
                                row_index, column_name] - theta*pooled_y_group_mean
                            
                #Carve out the y and X matrices:
                re_y=pooled_y_with_unit_name[list(pooled_y_with_unit_name.columns[1:])]
                re_X=pooled_X_with_unit_name[list(pooled_X_with_unit_name.columns[1:])]
                re_X= sm.add_constant(re_X)
                #Build and train the model
                re_model_s = sm.OLS(endog=re_y, exog=re_X)
                re_model_results_s = re_model_s.fit()
                if summary == True:
                    return re_model_results_s.summary()
                else:
                    return re_model_results_s
    elif g== True:
                k = len(X_re_s.columns) + 1

                df_with_dummies = df_econ_copy.copy()
                entity_dummies_ols = pd.get_dummies(df_eco_copy.index.get_level_values('port_id'), drop_first=True, prefix='port').astype(int)
                df_with_dummies = df_with_dummies.join(entity_dummies_ols)
                
                lsdv_expr = 'Return' + ' ~ '
                i = 0
                for X_var_name in X_re_g:
                    if i > 0:
                        lsdv_expr = lsdv_expr + ' + ' + X_var_name
                    else:
                        lsdv_expr = lsdv_expr + X_var_name
                    i = i + 1
                for dummy_name in unit_names[1:]:
                    lsdv_expr = lsdv_expr + ' + ' + dummy_name
            
                lsdv_model = smf.ols(formula=lsdv_expr, data=df_with_dummies)
                lsdv_model_results = lsdv_model.fit()

                
                #Calculate sigma-square-epsilon
                sigma2_epsilon = lsdv_model_results.ssr/(n*T-(n+k+1))
                # print('sigma2_epsilon = ' + str(sigma2_epsilon))

                #Calculate sigma-square-pooled
                sigma2_pooled = sm.OLS(endog=y_pols, exog=X_re_g).fit().ssr/(n*T-(k+1))
                # print('sigma2_pooled = ' + str(sigma2_pooled))

                #Calculate sigma-square-u
                sigma2_u = sigma2_pooled - sigma2_epsilon
                # print('sigma2_u = ' + str(sigma2_u))

                #Calculate theta
                theta = 1 - math.sqrt(sigma2_epsilon/(sigma2_epsilon + T*sigma2_u))
                # print('theta = ' + str(theta))
                
                
                df_eco_copy_means = df_econ_copy.groupby('port_id').mean()

                df_eco_copy_means['const'] = 1.0
                
                # print(df_eco_copy_means)
                
                
                # #Prepare the data set for mean centering the y and X columns:
                pooled_y_with_unit_name = pd.concat([df_econ_copy['port_id'], y_pols], axis=1)
                pooled_X_with_unit_name = pd.concat([df_econ_copy['port_id'], X_re_g], axis=1)

                #Center each X value using the θ-scaled group-specific mean:
                unit_name = ''
                for row_index, row in pooled_X_with_unit_name.iterrows():
                    for column_name, cell_value in row.items():
                        if column_name == 'port_id':
                            unit_name = pooled_X_with_unit_name.at[row_index, column_name]
                        else:
                            pooled_X_group_mean = df_eco_copy_means.loc[unit_name][column_name]
                            pooled_X_with_unit_name.at[row_index, column_name] = pooled_X_with_unit_name.at[
                                row_index, column_name] - theta*pooled_X_group_mean

                #Center each y value using the θ-scaled group-specific mean:
                for row_index, row in pooled_y_with_unit_name.iterrows():
                    for column_name, cell_value in row.items():
                        if column_name == 'port_id':
                            unit_name = pooled_y_with_unit_name.at[row_index, column_name]
                        else:
                            pooled_y_group_mean = df_eco_copy_means.loc[unit_name][column_name]
                            pooled_y_with_unit_name.at[row_index, column_name] = pooled_y_with_unit_name.at[
                                row_index, column_name] - theta*pooled_y_group_mean
                            
                #Carve out the y and X matrices:
                re_y=pooled_y_with_unit_name[list(pooled_y_with_unit_name.columns[1:])]
                re_X=pooled_X_with_unit_name[list(pooled_X_with_unit_name.columns[1:])]
                re_X= sm.add_constant(re_X)
                
                #Build and train the model
                re_model_g = sm.OLS(endog=re_y, exog=re_X)
                re_model_results_g = re_model_g.fit()
                if summary == True:
                    return re_model_results_g.summary()
                else:
                    return re_model_results_g
    elif all==True:
                k = len(X_re_all.columns) + 1

                df_with_dummies = df_econ_copy.copy()
                entity_dummies_ols = pd.get_dummies(df_eco_copy.index.get_level_values('port_id'), drop_first=True, prefix='port').astype(int)
                df_with_dummies = df_with_dummies.join(entity_dummies_ols)
                
                lsdv_expr = 'Return' + ' ~ '
                i = 0
                for X_var_name in X_pols_all:
                    if i > 0:
                        lsdv_expr = lsdv_expr + ' + ' + X_var_name
                    else:
                        lsdv_expr = lsdv_expr + X_var_name
                    i = i + 1
                for dummy_name in unit_names[1:]:
                    lsdv_expr = lsdv_expr + ' + ' + dummy_name
                
                lsdv_model = smf.ols(formula=lsdv_expr, data=df_with_dummies)
                lsdv_model_results = lsdv_model.fit()

            
                #Calculate sigma-square-epsilon
                sigma2_epsilon = lsdv_model_results.ssr/(n*T-(n+k+1))
                # print('sigma2_epsilon = ' + str(sigma2_epsilon))

                #Calculate sigma-square-pooled
                sigma2_pooled = sm.OLS(endog=y_pols, exog=X_re_all).fit().ssr/(n*T-(k+1))
                # print('sigma2_pooled = ' + str(sigma2_pooled))

                #Calculate sigma-square-u
                sigma2_u = sigma2_pooled - sigma2_epsilon
                # print('sigma2_u = ' + str(sigma2_u))

                #Calculate theta
                theta = 1 - math.sqrt(sigma2_epsilon/(sigma2_epsilon + T*sigma2_u))
                # print('theta = ' + str(theta))
                
                
                df_eco_copy_means = df_econ_copy.groupby('port_id').mean()

                df_eco_copy_means['const'] = 1.0
                
                # print(df_eco_copy_means)
                
                
                # #Prepare the data set for mean centering the y and X columns:
                pooled_y_with_unit_name = pd.concat([df_econ_copy['port_id'], y_pols], axis=1)
                pooled_X_with_unit_name = pd.concat([df_econ_copy['port_id'], X_pols_all], axis=1)

                #Center each X value using the θ-scaled group-specific mean:
                unit_name = ''
                for row_index, row in pooled_X_with_unit_name.iterrows():
                    for column_name, cell_value in row.items():
                        if column_name == 'port_id':
                            unit_name = pooled_X_with_unit_name.at[row_index, column_name]
                        else:
                            pooled_X_group_mean = df_eco_copy_means.loc[unit_name][column_name]
                            pooled_X_with_unit_name.at[row_index, column_name] = pooled_X_with_unit_name.at[
                                row_index, column_name] - theta*pooled_X_group_mean

                #Center each y value using the θ-scaled group-specific mean:
                for row_index, row in pooled_y_with_unit_name.iterrows():
                    for column_name, cell_value in row.items():
                        if column_name == 'port_id':
                            unit_name = pooled_y_with_unit_name.at[row_index, column_name]
                        else:
                            pooled_y_group_mean = df_eco_copy_means.loc[unit_name][column_name]
                            pooled_y_with_unit_name.at[row_index, column_name] = pooled_y_with_unit_name.at[
                                row_index, column_name] - theta*pooled_y_group_mean
                            
                #Carve out the y and X matrices:
                re_y=pooled_y_with_unit_name[list(pooled_y_with_unit_name.columns[1:])]
                re_X=pooled_X_with_unit_name[list(pooled_X_with_unit_name.columns[1:])]
                re_X= sm.add_constant(re_X)

                #Build and train the model
                re_model_all = sm.OLS(endog=re_y, exog=re_X)
                re_model_results_all = re_model_all.fit()
                if summary == True:
                    return re_model_results_all.summary()
                else:
                    return re_model_results_all
                  
    else: print('Set one of the parameters to True')

In [13]:
RE_model(df_eco_copy, df_econ_copy, summary=True, e=False, s=False, g=False, all=True)

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                 Return   R-squared:                       0.339
Model:                            OLS   Adj. R-squared:                  0.323
Method:                 Least Squares   F-statistic:                     20.70
Date:                Sun, 28 May 2023   Prob (F-statistic):           6.85e-11
Time:                        22:22:37   Log-Likelihood:                 34.543
No. Observations:                 125   AIC:                            -61.09
Df Residuals:                     121   BIC:                            -49.77
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const         -0.0613      0.152     -0.404      0.687      -0.362       0.239
E_Score       -0.6894      0.148     -4.662      0.000      -0.982      -0.397
S_Score        0.5771      0.159      3.638      0.000       0.263       0.891
G_Score        1.1443      0.365      3.134      0.002       0.421       1.867
==============================================================================
Omnibus:                       21.753   Durbin-Watson:                   0.907
Prob(Omnibus):                  0.000   Jarque-Bera (JB):               28.076
Skew:                           0.963   Prob(JB):                     8.01e-07
Kurtosis:                       4.295   Cond. No.                         39.4
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [20]:
RE_model(df_eco_copy, df_econ_copy, summary=True, e=True, s=False, g=False, all=False)

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                 Return   R-squared:                       0.026
Model:                            OLS   Adj. R-squared:                  0.018
Method:                 Least Squares   F-statistic:                     3.321
Date:                Sun, 28 May 2023   Prob (F-statistic):             0.0708
Time:                        22:30:07   Log-Likelihood:                 37.928
No. Observations:                 125   AIC:                            -71.86
Df Residuals:                     123   BIC:                            -66.20
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.1640      0.038      4.336      0.000       0.089       0.239
E_Score        0.1558      0.085      1.822      0.071      -0.013       0.325
==============================================================================
Omnibus:                       30.553   Durbin-Watson:                   1.027
Prob(Omnibus):                  0.000   Jarque-Bera (JB):               48.863
Skew:                           1.158   Prob(JB):                     2.45e-11
Kurtosis:                       5.004   Cond. No.                         6.18
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [22]:
RE_model(df_eco_copy, df_econ_copy, summary=True, e=False, s=True, g=False, all=False)

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                 Return   R-squared:                       0.121
Model:                            OLS   Adj. R-squared:                  0.114
Method:                 Least Squares   F-statistic:                     16.99
Date:                Sun, 28 May 2023   Prob (F-statistic):           6.86e-05
Time:                        22:31:28   Log-Likelihood:                 39.643
No. Observations:                 125   AIC:                            -75.29
Df Residuals:                     123   BIC:                            -69.63
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.1405      0.037      3.822      0.000       0.068       0.213
S_Score        0.4094      0.099      4.121      0.000       0.213       0.606
==============================================================================
Omnibus:                       23.767   Durbin-Watson:                   1.007
Prob(Omnibus):                  0.000   Jarque-Bera (JB):               32.035
Skew:                           1.014   Prob(JB):                     1.11e-07
Kurtosis:                       4.428   Cond. No.                         6.97
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [23]:
RE_model(df_eco_copy, df_econ_copy, summary=True, e=False, s=False, g=True, all=False)

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                 Return   R-squared:                       0.120
Model:                            OLS   Adj. R-squared:                  0.113
Method:                 Least Squares   F-statistic:                     16.74
Date:                Sun, 28 May 2023   Prob (F-statistic):           7.70e-05
Time:                        22:31:46   Log-Likelihood:                 34.913
No. Observations:                 125   AIC:                            -65.83
Df Residuals:                     123   BIC:                            -60.17
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const         -0.0194      0.085     -0.228      0.820      -0.188       0.149
G_Score        0.7348      0.180      4.091      0.000       0.379       1.090
==============================================================================
Omnibus:                       13.761   Durbin-Watson:                   0.943
Prob(Omnibus):                  0.001   Jarque-Bera (JB):               15.381
Skew:                           0.714   Prob(JB):                     0.000457
Kurtosis:                       3.956   Cond. No.                         13.3
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""